In [59]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from matplotlib.colors import Normalize, ListedColormap
from matplotlib.cm import ScalarMappable
import ast
import re


In [37]:
# first load tables

domain_table = pd.read_csv('/Volumes/PGH-Backup/domains/IPS/all_IPS_results.tsv', sep='\t', header=None)

display(domain_table, domain_table.shape)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Amidase_UniRef100_UPI00068D388E|798|g__Bacillu...,5e26d200a96cb5f9ea2077e03b21700c,266,Pfam,PF01520,N-acetylmuramoyl-L-alanine amidase,58,256,9.300000e-42,T,25-07-2024,IPR002508,"N-acetylmuramoyl-L-alanine amidase, catalytic ..."
1,Amidase_UniRef100_UPI0006B0624C|678|g__Bacillu...,c9e01160fcbeb852dc6e8f1ed1eabc2d,226,Pfam,PF01520,N-acetylmuramoyl-L-alanine amidase,6,170,1.300000e-26,T,25-07-2024,IPR002508,"N-acetylmuramoyl-L-alanine amidase, catalytic ..."
2,Amidase_UniRef100_UPI0006B0624C|678|g__Bacillu...,c9e01160fcbeb852dc6e8f1ed1eabc2d,226,Pfam,PF05036,SPOR domain,192,223,1.100000e-06,T,25-07-2024,IPR007730,Sporulation-like domain
3,Amidase_UniRef100_UPI000C9E9FDF|1716|g__Vibrio...,4649943be98becffba027e8bbd34a134,572,Pfam,PF01520,N-acetylmuramoyl-L-alanine amidase,169,388,1.600000e-43,T,25-07-2024,IPR002508,"N-acetylmuramoyl-L-alanine amidase, catalytic ..."
4,Amidase_UniRef100_UPI000C9E9FDF|1716|g__Vibrio...,4649943be98becffba027e8bbd34a134,572,Pfam,PF11741,AMIN domain,33,135,4.100000e-17,T,25-07-2024,IPR021731,AMIN domain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107338,UC118_WP_160944966.1|1551|g__Ligilactobacillus...,e016b1d9b174ec66992850fbef4c98d1,517,Pfam,PF01476,LysM domain,34,77,2.100000e-11,T,25-07-2024,IPR018392,LysM domain
1107339,UC118_WP_160998332.1|1515|g__Ligilactobacillus...,27d421e12a913112ee1d18a5bd385322,505,Pfam,PF01476,LysM domain,94,136,6.800000e-15,T,25-07-2024,IPR018392,LysM domain
1107340,UC118_WP_160998332.1|1515|g__Ligilactobacillus...,27d421e12a913112ee1d18a5bd385322,505,Pfam,PF01476,LysM domain,34,77,2.000000e-11,T,25-07-2024,IPR018392,LysM domain
1107341,UC118_WP_160998332.1|1515|g__Ligilactobacillus...,27d421e12a913112ee1d18a5bd385322,505,Pfam,PF01476,LysM domain,252,292,7.200000e-09,T,25-07-2024,IPR018392,LysM domain


(1107343, 13)

In [38]:
# Group by the enzyme name and aggregate the domain column as a list
grouped_df = domain_table.groupby(0)[5].apply(list).reset_index()


In [39]:
display(grouped_df)

,0,5
0,Amidase_UniRef100_A0A009ES59|570|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase]
1,Amidase_UniRef100_A0A009FUX6|744|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase]
2,Amidase_UniRef100_A0A009H4S4|567|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase]
3,Amidase_UniRef100_A0A009HT94|567|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase]
4,Amidase_UniRef100_A0A009L0R9|591|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase]
...,...,...
707784,UC118_WP_253005939.1|1548|g__Ligilactobacillus...,"[NlpC/P60 family, LysM domain, LysM domain, Ly..."
707785,UC118_WP_255820014.1|1563|g__Ligilactobacillus...,"[LysM domain, LysM domain, LysM domain, NlpC/P..."
707786,UC118_WP_263296879.1|1554|g__Ligilactobacillus...,"[NlpC/P60 family, LysM domain, LysM domain, Ly..."
707787,UC118_WP_263297069.1|1554|g__Ligilactobacillus...,"[LysM domain, LysM domain, LysM domain, NlpC/P..."


In [40]:
grouped_df_clean = grouped_df

In [41]:
grouped_df_clean[0] = grouped_df_clean[0].str.split('|').str[0]

display(grouped_df_clean)

,0,5
0,Amidase_UniRef100_A0A009ES59,[N-acetylmuramoyl-L-alanine amidase]
1,Amidase_UniRef100_A0A009FUX6,[N-acetylmuramoyl-L-alanine amidase]
2,Amidase_UniRef100_A0A009H4S4,[N-acetylmuramoyl-L-alanine amidase]
3,Amidase_UniRef100_A0A009HT94,[N-acetylmuramoyl-L-alanine amidase]
4,Amidase_UniRef100_A0A009L0R9,[N-acetylmuramoyl-L-alanine amidase]
...,...,...
707784,UC118_WP_253005939.1,"[NlpC/P60 family, LysM domain, LysM domain, Ly..."
707785,UC118_WP_255820014.1,"[LysM domain, LysM domain, LysM domain, NlpC/P..."
707786,UC118_WP_263296879.1,"[NlpC/P60 family, LysM domain, LysM domain, Ly..."
707787,UC118_WP_263297069.1,"[LysM domain, LysM domain, LysM domain, NlpC/P..."


In [42]:
grouped_df_clean.to_csv('/Volumes/PGH-Backup/domains/IPS/all_IPS_results_grouped.tsv', sep='\t', index=False, header=False)

**Begin Merging with cluster table**

In [43]:
## Temporary swap to a windows enviorment, paths changed accordingly
# grouped_df_clean = pd.read_csv("E:\\domains\\IPS\\all_IPS_results_grouped.tsv", sep='\t', header=None)
grouped_df_clean = pd.read_csv("/Volumes/PGH-Backup/domains/IPS/all_IPS_results_grouped.tsv", sep='\t', header=None)

display(grouped_df_clean)

,0,1
0,Amidase_UniRef100_A0A009ES59,['N-acetylmuramoyl-L-alanine amidase']
1,Amidase_UniRef100_A0A009FUX6,['N-acetylmuramoyl-L-alanine amidase']
2,Amidase_UniRef100_A0A009H4S4,['N-acetylmuramoyl-L-alanine amidase']
3,Amidase_UniRef100_A0A009HT94,['N-acetylmuramoyl-L-alanine amidase']
4,Amidase_UniRef100_A0A009L0R9,['N-acetylmuramoyl-L-alanine amidase']
...,...,...
707784,UC118_WP_253005939.1,"['NlpC/P60 family', 'LysM domain', 'LysM domai..."
707785,UC118_WP_255820014.1,"['LysM domain', 'LysM domain', 'LysM domain', ..."
707786,UC118_WP_263296879.1,"['NlpC/P60 family', 'LysM domain', 'LysM domai..."
707787,UC118_WP_263297069.1,"['LysM domain', 'LysM domain', 'LysM domain', ..."


In [44]:
grouped_df_clean.rename(columns={0: 'FullIdentifier', 1: 'Domains'}, inplace=True)

In [45]:
grouped_df_clean['Uniref'] = grouped_df_clean['FullIdentifier'].str.split('_').str[2]

grouped_df_clean['Enzyme'] = grouped_df_clean['FullIdentifier'].str.split('_').str[0]

display(grouped_df_clean)

,FullIdentifier,Domains,Uniref,Enzyme
0,Amidase_UniRef100_A0A009ES59,['N-acetylmuramoyl-L-alanine amidase'],A0A009ES59,Amidase
1,Amidase_UniRef100_A0A009FUX6,['N-acetylmuramoyl-L-alanine amidase'],A0A009FUX6,Amidase
2,Amidase_UniRef100_A0A009H4S4,['N-acetylmuramoyl-L-alanine amidase'],A0A009H4S4,Amidase
3,Amidase_UniRef100_A0A009HT94,['N-acetylmuramoyl-L-alanine amidase'],A0A009HT94,Amidase
4,Amidase_UniRef100_A0A009L0R9,['N-acetylmuramoyl-L-alanine amidase'],A0A009L0R9,Amidase
...,...,...,...,...
707784,UC118_WP_253005939.1,"['NlpC/P60 family', 'LysM domain', 'LysM domai...",253005939.1,UC118
707785,UC118_WP_255820014.1,"['LysM domain', 'LysM domain', 'LysM domain', ...",255820014.1,UC118
707786,UC118_WP_263296879.1,"['NlpC/P60 family', 'LysM domain', 'LysM domai...",263296879.1,UC118
707787,UC118_WP_263297069.1,"['LysM domain', 'LysM domain', 'LysM domain', ...",263297069.1,UC118


In [46]:
grouped_df_clean.drop_duplicates(subset='Uniref', inplace=True)

display(grouped_df_clean)


,FullIdentifier,Domains,Uniref,Enzyme
0,Amidase_UniRef100_A0A009ES59,['N-acetylmuramoyl-L-alanine amidase'],A0A009ES59,Amidase
1,Amidase_UniRef100_A0A009FUX6,['N-acetylmuramoyl-L-alanine amidase'],A0A009FUX6,Amidase
2,Amidase_UniRef100_A0A009H4S4,['N-acetylmuramoyl-L-alanine amidase'],A0A009H4S4,Amidase
3,Amidase_UniRef100_A0A009HT94,['N-acetylmuramoyl-L-alanine amidase'],A0A009HT94,Amidase
4,Amidase_UniRef100_A0A009L0R9,['N-acetylmuramoyl-L-alanine amidase'],A0A009L0R9,Amidase
...,...,...,...,...
707784,UC118_WP_253005939.1,"['NlpC/P60 family', 'LysM domain', 'LysM domai...",253005939.1,UC118
707785,UC118_WP_255820014.1,"['LysM domain', 'LysM domain', 'LysM domain', ...",255820014.1,UC118
707786,UC118_WP_263296879.1,"['NlpC/P60 family', 'LysM domain', 'LysM domai...",263296879.1,UC118
707787,UC118_WP_263297069.1,"['LysM domain', 'LysM domain', 'LysM domain', ...",263297069.1,UC118


In [47]:
# read in cluster map
# cluster_map = pd.read_csv("E:\\clustering\\newest_cluster_maps\\catted_maps.tsv", sep='\t', index_col=0, header=None)
cluster_map = pd.read_csv("/Volumes/PGH-Backup/clustering/newest_cluster_maps/catted_maps.tsv", sep='\t', index_col=0, header=None)

display(cluster_map)

,1,2,3
0,,,
0,A0A432IFZ8,A0A432IFZ8,A0A534TJ56
1,A0A369Y4I6,A0A432IFZ8,A0A534TJ56
2,E3HA38,A0A432IFZ8,A0A534TJ56
3,UPI0004050AE8,A0A369PUK8,A0A455SSC1
4,A0A924USN5,A0A369PUK8,A0A455SSC1
...,...,...,...
331605,R5VGU4,R5VGU4,R5VGU4
331606,T0DIM4,T0DIM4,A0A5N7MBX0
331607,UPI000EA3697F,T0DIM4,A0A5N7MBX0


In [48]:
cluster_map.rename(columns={1: 'unclustered', 2: 'mmseqs', 3: 'foldseek'}, inplace=True)

cluster_map.drop_duplicates(subset='unclustered', inplace=True)

display(cluster_map)

,unclustered,mmseqs,foldseek
0,,,
0,A0A432IFZ8,A0A432IFZ8,A0A534TJ56
1,A0A369Y4I6,A0A432IFZ8,A0A534TJ56
2,E3HA38,A0A432IFZ8,A0A534TJ56
3,UPI0004050AE8,A0A369PUK8,A0A455SSC1
4,A0A924USN5,A0A369PUK8,A0A455SSC1
...,...,...,...
331605,R5VGU4,R5VGU4,R5VGU4
331606,T0DIM4,T0DIM4,A0A5N7MBX0
331607,UPI000EA3697F,T0DIM4,A0A5N7MBX0


In [49]:
# store unique enzyme names to loop through
enzymes = grouped_df_clean['Enzyme'].unique()
print(enzymes)

['Amidase' 'DD-carboxypeptidase' 'DD-endopeptidase' 'DL-endopeptidase'
 'Diadenylate' 'Glucosaminidase' 'LD-carboxypeptidase' 'LD-endopeptidase'
 'Muramidase' 'SagA' 'UC118']


In [89]:
test = ["['Chitinase class I', 'Putative peptidoglycan binding domain']", "['Chitinase class I', 'LysM domain']"]

In [90]:
print(test, type(test), type(test[0]))

["['Chitinase class I', 'Putative peptidoglycan binding domain']", "['Chitinase class I', 'LysM domain']"] <class 'list'> <class 'str'>


In [85]:
test = [ast.literal_eval(x) for x in test]

print(type(test), type(test[0]))

flat_test = [
    item
    for sublist in test
    for item in sublist
]

print(flat_test)


<class 'list'> <class 'list'>
['Chitinase class I', 'Putative peptidoglycan binding domain', 'Chitinase class I', 'LysM domain']


In [86]:
def clean_and_convert(domain_string):
    domain_string = [ast.literal_eval(x) for x in domain_string]
    
    domain_string = [
        item
        for sublist in domain_string
        for item in sublist
    ]
    
    return domain_string
    

In [91]:
# Subset by enzymes and merge with cluster map

for enzyme in enzymes:
    grouped_df_clean_subset = grouped_df_clean[grouped_df_clean['Enzyme'] == enzyme]
    merged_df = pd.merge(grouped_df_clean_subset, cluster_map, left_on='Uniref', right_on='unclustered', how='left')
    merged_df = merged_df.dropna(subset=['foldseek'])

    if merged_df.shape[0] > 0:
        grouped_merged_domain_cluster = merged_df.groupby('foldseek').agg({
                'Domains': list,    # Aggregate Domains into a list
                'Uniref': list,     # Aggregate Uniref into a list
                'Enzyme': set       # Aggregate Enzyme into a set (to remove duplicates)
            }).reset_index()
        
        grouped_merged_domain_cluster['member_count'] = grouped_merged_domain_cluster['Uniref'].apply(len)

        grouped_merged_domain_cluster['Domains'] = grouped_merged_domain_cluster['Domains'].apply(
            lambda x: clean_and_convert(x) if isinstance(x, list) else x)
        
        print(grouped_merged_domain_cluster.head())

        grouped_merged_domain_cluster.to_csv(f"/Volumes/PGH-Backup/domains/IPS/{enzyme}_IPS_results_grouped.tsv", sep='\t', index=False)

     foldseek                                            Domains  \
0  A0A017TEG0  [N-acetylmuramoyl-L-alanine amidase, N-acetylm...   
1  A0A024GY02  [Peptidase family M23, Peptidase family M23, B...   
2  A0A024P1R2  [SPOR domain, SPOR domain, N-acetylmuramoyl-L-...   
3  A0A024YV93                                       [PKD domain]   
4  A0A060CJU1  [N-acetylmuramoyl-L-alanine amidase, N-acetylm...   

                                              Uniref     Enzyme  member_count  
0  [A0A017TEG0, A0A540WW64, A0A5C5VU41, A0A7Y7CBT...  {Amidase}            11  
1  [A0A024GX01, A0A024GY02, A0A096CCQ9, A0A1C6JAB...  {Amidase}            76  
2  [A0A024P1R2, A0A1H0QRP2, A0A1I3W791, A0A6V8PBU...  {Amidase}            19  
3                                       [A0A024YT43]  {Amidase}             1  
4         [A0A060CJU1, UPI0005C55A47, UPI001F1DD695]  {Amidase}             3  
     foldseek                                            Domains  \
0  A0A024FL54  [Beta-lactamase, Domain of u

**Generate Network Plots**

In [107]:
def calculate_domain_similarity(subset_df):

    subset_df = pd.read_csv(subset_df, sep='\t')
    
    # Initialize to track how many proteins have each domain in each cluster
    domain_occurrences = defaultdict(lambda: defaultdict(int))

    # Iterate over the dataframe to count domain presence per protein
    for index, row in merged_df.iterrows():
        cluster_id = row['foldseek']
        domains = row['Domains']

        # Check if 'domains' is a valid non-empty list or string
        if isinstance(domains, str) and domains.strip() != '':
            try:
                # Safely evaluate the string into a list
                unique_domains = set(eval(domains))
            except:
                # Skip any domains that can't be evaluated
                continue
        elif isinstance(domains, (list, tuple, np.ndarray)) and len(domains) > 0:
            unique_domains = set(domains)
        else:
            # Skip if 'domains' is None, NaN, empty, or not a valid type
            continue

        # Count each domain in the set
        for domain in unique_domains:
            domain_occurrences[cluster_id][domain] += 1

    # Calculate the percentage of proteins with each domain in each cluster
    domain_percentages = {}
    for cluster, domains_dict in domain_occurrences.items():
        member_count = merged_df.loc[merged_df['foldseek'] == cluster, 'member_count'].values[0]
        domain_percentages[cluster] = {domain: count / member_count for domain, count in domains_dict.items()}

    # Convert domain percentages to a matrix for similarity calculation
    # Create a list of all unique domains across all clusters
    all_domains = set(domain for cluster_domains in domain_percentages.values() for domain in cluster_domains.keys())

    # Create a matrix of domain percentages for each cluster
    cluster_ids = list(domain_percentages.keys())
    domain_matrix = np.zeros((len(cluster_ids), len(all_domains)))

    # Mapping of cluster IDs and domain indices to facilitate matrix population
    cluster_idx_map = {cluster_id: idx for idx, cluster_id in enumerate(cluster_ids)}
    domain_idx_map = {domain: idx for idx, domain in enumerate(all_domains)}

    # Populate the matrix with domain percentages
    for cluster_id, domains_dict in domain_percentages.items():
        for domain, percentage in domains_dict.items():
            cluster_idx = cluster_idx_map[cluster_id]
            domain_idx = domain_idx_map[domain]
            domain_matrix[cluster_idx, domain_idx] = percentage

    # Calculate cosine similarity between clusters based on domain matrix
    cosine_sim = cosine_similarity(domain_matrix)

    # Convert the similarity matrix to edge list for significant similarities
    # We consider a similarity significant if it's above 0.1 (can adjust)
    significant_similarity_threshold = 0.1
    significant_edges = []
    for i in range(len(cluster_ids)):
        for j in range(i+1, len(cluster_ids)):
            if cosine_sim[i, j] > significant_similarity_threshold:
                significant_edges.append((cluster_ids[i], cluster_ids[j], cosine_sim[i, j]))
        
    return significant_edges


In [108]:
def plot_network(enzyme_type, subset_df, significant_edges, association_table=None, title=None):
    """ Plot the network graph for each unique enzyme type. Adjust node colors based on
        the enzyme type and node sizes based on cluster sizes. """
    
    subset_df = pd.read_csv(subset_df, sep='\t')

    # Create the network graph
    G_adjusted_similarity = nx.Graph()

    # Subset cluster_ids and edges based on the enzyme type in subset_df
    cluster_ids_subset = subset_df['foldseek'].tolist()
    edges_subset = [(u, v, w) for u, v, w in significant_edges if u in cluster_ids_subset and v in cluster_ids_subset]

    # Add nodes (clusters) for the subset
    G_adjusted_similarity.add_nodes_from(cluster_ids_subset)

    # Add edges with weights based on cosine similarity for the subset
    G_adjusted_similarity.add_weighted_edges_from(edges_subset)

    # Check if association_table is provided
    if association_table is not None:
        # Make association dict
        association_dict = dict(zip(association_table['feature'], association_table['scaled_value']))

        # Define color scaling function
        def get_node_color(node):
            if node in association_dict:
                return association_dict[node]
            else:
                return None

        # Assign colors to nodes based on their association values
        node_values = [get_node_color(node) for node in G_adjusted_similarity.nodes()]
        abs_max_val = max(abs(val) for val in node_values if val is not None)
        min_val = -abs_max_val
        max_val = abs_max_val
        reversed_bwr = plt.cm.bwr.reversed()  # Reverse the colormap

        node_colors = [
            'grey' if value is None or -3 < value < 3 else reversed_bwr((value - min_val) / (max_val - min_val))
            for value in node_values   
        ]
    else:
        # Default all node colors to grey if association_table is not provided
        node_colors = ['red' for _ in G_adjusted_similarity.nodes()]

    # Get the cluster sizes from the 'member_count' column in the subset_df
    cluster_sizes = subset_df.set_index('foldseek')['member_count'].to_dict()

    # Normalize cluster sizes to a suitable range for node sizes in the graph
    min_size = 20
    max_size = 1000
    min_cluster_size = min(cluster_sizes.values())
    max_cluster_size = max(cluster_sizes.values())
    node_sizes = [
        ((cluster_sizes[node] - min_cluster_size) / (max_cluster_size - min_cluster_size) * (max_size - min_size) + min_size)
        if node in cluster_sizes else min_size
        for node in G_adjusted_similarity.nodes()
    ]

    # Visualize the adjusted network for the enzyme type
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G_adjusted_similarity, seed=42)  # For consistent layout

    # Draw the network
    nodes = nx.draw_networkx_nodes(G_adjusted_similarity, pos, node_color=node_colors, node_size=node_sizes, alpha=0.8)
    nx.draw_networkx_edges(G_adjusted_similarity, pos, alpha=0.5)
    nx.draw_networkx_labels(G_adjusted_similarity, pos, font_size=5, alpha=0.7)

    # Add title
    if title is not None:
        plt.title(f"{title} - {enzyme_type}")
    else:
        plt.title(f"Adjusted Network Graph for {enzyme_type}")
    plt.axis('off')

    # Add colorbar if association_table is provided
    if association_table is not None:
        # Create a ScalarMappable for the colorbar
        sm = ScalarMappable(cmap=reversed_bwr, norm=Normalize(vmin=min_val, vmax=max_val))
        sm.set_array([])  # Dummy array for ScalarMappable
        plt.colorbar(sm, label='Association Value')

    # Save the plot for each enzyme type
    plt.savefig(f'./{enzyme_type}_adjusted_network.png', dpi=600, bbox_inches='tight')

In [109]:
enzymes = ['Amidase', 'DD-carboxypeptidase', 'DD-endopeptidase', 'DL-endopeptidase', 
           'Glucosaminidase', 'LD-carboxypeptidase', 'LD-endopeptidase', 'Muramidase']

In [110]:
# Loop through each enzyme type and generate the corresponding plot

for enzyme in enzymes:
    path = f'/Volumes/PGH-Backup/domains/IPS/clustered/{enzyme}_IPS_results_grouped.tsv'
    
    significant_edges = calculate_domain_similarity(path)

    plot_network(
        enzyme_type=enzyme,
        subset_df=path,
        significant_edges=significant_edges,
        title=None
    )

ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required by check_pairwise_arrays.